In [1]:
%load_ext sql

In [2]:
%sql sqlite://

We can create tables use `CREATE TABLE` and drop them using `DROP TABLE`.

In [3]:
%%sql
CREATE TABLE Person(id text, personal text, family text);
CREATE TABLE Site(name text, lat real, long real);
CREATE TABLE Visited(id integer, site text, dated text);
CREATE TABLE Survey(taken integer, person text, quant text, reading real);

 * sqlite://
Done.
Done.
Done.
Done.


[]

In [4]:
%%sql
DROP TABLE Survey;

 * sqlite://
Done.


[]

We might want to put certain constraints on tables. Some constraints could include:
- Not allowing certain columns to be NULL
- Defining 1 or more columns as the `PRIMARY KEY`, which uniquely defines a row
- Referencing a column to another table we'll join with using a `FOREIGN KEY`

#### TODO
For example, let's look at our definition of `Survey`.

```sql
CREATE TABLE Survey(
    taken integer, 
    person text, 
    quant text, 
    reading real
);
```

- Which of these columns should not be allowed to be missing?
- What might be a good primary key?
- Do any of the columns reference other tables?

In [5]:
%%sql
CREATE TABLE Survey(
    taken   integer not null, -- where reading taken
    person  text,             -- may not know who took it
    quant   text not null,    -- the quantity measured
    reading real not null,    -- the actual reading
    primary key(taken, person, quant),    -- key is taken + person + quant
    foreign key(taken) references Visited(id),
    foreign key(person) references Person(id)
);

 * sqlite://
Done.


[]

## Updating data
Next let's put some data in our tables.

The syntax for this is:
```sql
INSERT INTO [Table Name]
VALUES (val1, val2, ...);
```

So, let's add some rows of data to `Site`:

 * sqlite://
1 rows affected.


[]

In [6]:
%%sql
INSERT INTO Site (name, lat, long) VALUES ('DR-1', -49.85, -128.57);
INSERT INTO Site (name, lat, long) VALUES ('DR-3', -47.15, -126.72);
INSERT INTO Site (name, lat, long) VALUES ('MSK-4', -48.87, -123.40);

 * sqlite://
1 rows affected.
1 rows affected.
1 rows affected.


[]

#### TODO
What would would happen if we changed `-123.40` to a string?

What would happen if we changed `-123.40` to `'abc.de'`?

In [7]:
%%sql
INSERT INTO Site (name, lat, long) VALUES ('MSK-4', -48.87, 'abc.de');

 * sqlite://
1 rows affected.


[]

#### TODO
What is wrong with this `INSERT` statement?
```sql
INSERT INTO Survey(person, quant, reading)
VALUES ('alec', 'temp', 0)
```

#### TODO
Insert the following rows into `Person`:
- Alec Chapman, id ABC
- Rebecca Barter, id RB

In [11]:
%%sql
INSERT INTO Person (
    id,
    personal,
    family
)
VALUES ('abc', 'Alec', 'Chapman'), 
    ('RB', 'Rebecca', 'Barter')

 * sqlite://
2 rows affected.


[]

#### TODO
Insert the following row into `Survey`:

- Taken: 1
- person: Rebecca Barter's ID
- quant: 'temp'
- reading: 0

In [18]:
%%sql
INSERT INTO Survey(taken, person, quant, reading)
VALUES (1, 'rb', 'temp', 0)

 * sqlite://
(sqlite3.IntegrityError) UNIQUE constraint failed: Survey.taken, Survey.person, Survey.quant
[SQL: INSERT INTO Survey(taken, person, quant, reading)
VALUES (1, 'rb', 'temp', 0)]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


We can also insert values from one table into another:

In [19]:
%%sql
CREATE TABLE JustLatLong(lat real, long real);

 * sqlite://
(sqlite3.OperationalError) table JustLatLong already exists
[SQL: CREATE TABLE JustLatLong(lat real, long real);]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [20]:
%%sql
INSERT INTO JustLatLong SELECT lat, long FROM Site;
SELECT * FROM JustLatLong

 * sqlite://
4 rows affected.
Done.


lat,long
-49.85,-128.57
-47.15,-126.72
-47.87,-122.4
-47.87,-122.4


We can change values in existing rows using the `UPDATE` keyword. So let's say we realized that the latitude and longitude were incorrect for site **MSK-4**. We can fix it using this command:

In [10]:
%%sql
UPDATE Site SET lat = -47.87, long = -122.40 WHERE name = 'MSK-4';

 * sqlite://
2 rows affected.


[]

We can delete rows using `DELETE`. This follows the syntax:

```sql
DELETE FROM table WHERE ...
```

#### TODO
For example, let's say we only wanted to keep sites in `JustLatLong` with a latitude > -49. 

In [21]:
%%sql
DELETE FROM JustLatLong
WHERE lat <= -49;

 * sqlite://
1 rows affected.


[]

In [22]:
%%sql
SELECT * FROM JustLatLong

 * sqlite://
Done.


lat,long
-47.15,-126.72
-47.87,-122.4
-47.87,-122.4


before we added both my own and Rebecca's names to `Person`, but only added survey measurement data for Rebecca. So let's take me off the expedition. How would we do that?

In [24]:
%%sql
DELETE 
FROM Person
WHERE id = 'abc';

 * sqlite://
1 rows affected.


[]

We could also removed Rebecca's name - but what would be a problem with this?

In [25]:
%%sql
DELETE 
FROM Person
WHERE id = 'RB';

 * sqlite://
1 rows affected.


[]

#### TODO
In the original dataset, modify the `Survey.person` column and replace all the NULL values with the string 'unknown'.